In [8]:
from Crypto.Util.Padding import pad, unpad
from Crypto.Cipher import AES
from hashlib import sha256

from ET_L23_qkd import QKD

bits = 32
qkd = QKD(bits)

alice_raw_key, bob_raw_key = qkd.generate_raw_key()
print(f"Alice raw key: {alice_raw_key}")
print(f"Bob raw key: {bob_raw_key}")

alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)

if len(alice_key) == 2:
    keys = list(set([ x + y for x, y in zip(alice_key[0], alice_key[1]) ]))
else:
    keys = list(set(alice_key[0]))

for key in keys:
    if key == bob_key:
        print("QKD completed!")
        print(f"Shared key: {key}")

        alice_key = sha256(key.encode()).digest()
        bob_key = sha256(bob_key.encode()).digest()

        MSG = b"H3xTEL{R3v3rS3_R3C0nc1l14710N_1s_P34K!}"
        cipher = AES.new(alice_key, AES.MODE_ECB)
        encrypted = cipher.encrypt(pad(MSG, 16))

        print(f"Alice encrypted message: {encrypted.hex()}")

        cipher = AES.new(bob_key, AES.MODE_ECB)
        decrypted = unpad(cipher.decrypt(encrypted), 16)

        print(f"Bob decrypted message: {decrypted.decode()}")

Alice raw key: 01000111100110101000101111101000
Bob raw key: 01110011100110100000100101111000
QKD completed!
Shared key: 011100011101111110100000101010011100000111111110100000001010011110101011110111111010010001011110011111000000110000000000101101101100010110010100111000110010011111101000100101010100100110010100000010011101110111010111111110000010000110101001000011011111110100111000100111000110011010011111101010110100010100111111110000000011000011011000100011011110011000101110001100000000010111000100110011100100010111010101100011010110000110000100100101100111010010110110101000001111001000100000011001010111010010001101110101100110101000110100010000010111110010011111010000010110001101000001111000100000101111000101001101011110110011101000000010010110110111101111001101011000011001001000111000111110100110011010001001010110111101001001001100100100111011111010110011001001010001101111010110000100010010111000110110011110000111101011010111110011001101100100100001110001100100001010000010101110001

In [ ]:
qubits = [8, 16, 32]

results = {}

for value in qubits:
    results[value] = [0, 0]
    
# file = open("success_probabilities.txt", "w")
# file.write(str(results))
# file.close()

In [13]:
from ET_L23_qkd import QKD

def get_success_probability(bits):
    file = open("success_probabilities.txt", "r")
    results = eval(file.read())
    file.close()
    
    total = results[bits][1]
    success = results[bits][0]
    actual = 0
    for i in range(100000):
        qkd = QKD(bits)
        alice_raw_key, bob_raw_key = qkd.generate_raw_key()
        alice_key, bob_key = qkd.generate_sifting_key(alice_raw_key, bob_raw_key)
        if len(alice_key) == 2:
            keys = list(set([ x + y for x, y in zip(alice_key[0], alice_key[1]) ]))
        elif len(alice_key) == 1:
            keys = list(set(alice_key[0]))
        else:
            total += 1
            results[bits][1] += 1
            continue
        
        for key in keys:
            if key == bob_key:
                success += 1
                results[bits][0] += 1
                
        total += 1
        results[bits][1] += 1
        
        if (abs(actual - (success / total)) != 0) and (abs(actual - (success / total)) < 1e-4) and i > 10000:
            actual = success / total
            print(f"Success probability: {success / total}")
            break

        actual = success / total

        file = open("success_probabilities.txt", "w")
        file.write(str(results))
        file.close()

In [14]:
# 8 qubits send
get_success_probability(8)

Success probability: 0.2669932027189124


In [15]:
# 16 qubits send
get_success_probability(16)

Success probability: 0.8715256948610278


In [16]:
# 32 qubits send
get_success_probability(32)

Success probability: 0.9986002799440112
